# Python抓取静态网页数据


作者：上海交通大学 安泰经济与管理学院 周志中

2018-09-14


## 使用BeautifulSoup解析获取中国银行外汇牌价数据

BeautifulSoup是一个可以从HTML或XML文件中提取数据的Python库。它能够通过你喜欢的转换器实现惯用的文档导航、查找、修改文档的方式。Beautiful Soup会帮你节省数小时甚至数天的工作时间。BeautifulSoup 4.4.0的说明文档在[这里](https://beautifulsoup.readthedocs.io/zh_CN/v4.4.0)。还有对BeautifulSoup的[简明介绍](https://www.cnblogs.com/zhaof/p/6930955.html)。首先读入网页源代码存入html变量。

In [1]:
from bs4 import BeautifulSoup
import urllib.request

url='http://www.boc.cn/sourcedb/whpj/'
resp=urllib.request.urlopen(url)
html=resp.read().decode("utf-8")

接下来使用BeautifulSoup解析网页。

In [2]:
bs = BeautifulSoup(html, "lxml")

在Firefox浏览器中打开外汇牌价网页，在网页上点击右键，然后点击Inspect Element（或者按“Q”键），这时候弹出Inspector窗口。可以发现标签为<tr>与</tr>之间的内容就是外汇牌价表的行内容。根据此信息提取所有标签为<tr>的内容。

In [3]:
contents = bs.find_all('tr')

下面的代码显示，提取出来的内容总共有31条，不过我们需要的数据从第2个元素开始，到第29个元素结束（对应的下标分别是1和28）。

In [4]:
print(len(contents))
print(contents[1])
print(contents[28])

31
<tr>
<th>货币名称</th>
<th>现汇买入价</th>
<th>现钞买入价</th>
<th>现汇卖出价</th>
<th>现钞卖出价</th>
<th>中行折算价</th>
<th>发布日期</th>
<th>发布时间</th>
</tr>
<tr>
<td>南非兰特</td>
<td>45.81</td>
<td>42.3</td>
<td>46.13</td>
<td>49.64</td>
<td>46.27</td>
<td>2018-09-15</td>
<td>21:36:54</td>
</tr>


contents[1]是表格头，表格头内容放在<th>标签中。contents[2]到contents[28]存放表格数据，内容放在<td>标签中。下面接着继续提取这些标签当中的内容。

In [5]:
print(contents[1].find_all('th'))
print(contents[2].find_all('td'))

[<th>货币名称</th>, <th>现汇买入价</th>, <th>现钞买入价</th>, <th>现汇卖出价</th>, <th>现钞卖出价</th>, <th>中行折算价</th>, <th>发布日期</th>, <th>发布时间</th>]
[<td>阿联酋迪拉姆</td>, <td></td>, <td>180.36</td>, <td></td>, <td>193.44</td>, <td>186.14</td>, <td>2018-09-15</td>, <td>21:36:54</td>]


下面的语句就去掉了标签<th>而提取出标签<th>中的内容。

In [6]:
thead = [];
for elem in contents[1].find_all('th'):
    thead.append(elem.string)
print(thead)

['货币名称', '现汇买入价', '现钞买入价', '现汇卖出价', '现钞卖出价', '中行折算价', '发布日期', '发布时间']


下面的语句就去掉了标签<td>而提取出标签<td>中的内容。

In [7]:
trow = [];
for elem in contents[2].find_all('td'):
    trow.append(elem.string)
print(trow)

['阿联酋迪拉姆', None, '180.36', None, '193.44', '186.14', '2018-09-15', '21:36:54']


有了以上准备之后，我们可以提取出表格当中的所有数据，放在一个数据框df当中。

In [8]:
import pandas as pd
df = pd.DataFrame(columns=thead)
for i in range(2,29):
    trow = [];
    for elem in contents[i].find_all('td'):
        trow.append(elem.string)
    df.loc[i-2]=trow
print(df)

      货币名称   现汇买入价   现钞买入价   现汇卖出价    现钞卖出价   中行折算价        发布日期      发布时间
0   阿联酋迪拉姆    None  180.36    None   193.44  186.14  2018-09-15  21:36:54
1    澳大利亚元  489.58  474.37  493.18   494.26  491.65  2018-09-15  21:36:54
2    巴西里亚尔    None  157.99    None    172.8  162.64  2018-09-15  21:36:54
3     加拿大元  525.05  508.47  528.92   530.08  525.93  2018-09-15  21:36:54
4     瑞士法郎  707.23   685.4  712.19   713.97  707.83  2018-09-15  21:36:54
5     丹麦克朗  106.54  103.25   107.4   107.61  107.17  2018-09-15  21:36:54
6       欧元  795.37  770.66  801.24   802.83  799.49  2018-09-15  21:36:54
7       英镑  894.38  866.59  900.96   902.94  896.66  2018-09-15  21:36:54
8       港币   87.32   86.62   87.67    87.67   87.11  2018-09-15  21:36:54
9     印尼卢比    None  0.0448    None    0.048  0.0461  2018-09-15  21:36:54
10    印度卢比    None  8.9776    None  10.1236  9.4978  2018-09-15  21:36:54
11      日元  6.1083  5.9185  6.1532   6.1532  6.1042  2018-09-15  21:36:54
12     韩国元  0.6096  0.5881  0.6144   0

## 使用Selenium抓取中国银行外汇牌价数据

在中国银行官方网站找到[外汇牌价数据URL地址](http://www.boc.cn/sourcedb/whpj)。在Firefox浏览器中打开网页，在外汇牌价列表中点击右键，然后点击Inspect Element（或者按“Q”键），这时候弹出Inspector窗口。由于外汇牌价数据放在一个表（table）当中，我们可以找<tbody>标签，鼠标移到该标签时表格数据变蓝色，意味着选取该标签可以提取表格所有数据。在代码行上点击右键，然后选择Copy，CSS Selector，将CSS Selector定位地址拷贝到剪贴板。这个定位地址是：.publish > div:nth-child(3) > table:nth-child(2) > tbody:nth-child(1)。有了CSS Selector定位地址，就可以提取该标签下的所有取值信息。使用Selenium定位网页元素的其他方法可在[Locating Elements](https://selenium-python.readthedocs.io/locating-elements.html)找到。

In [9]:
from selenium import webdriver

driver = webdriver.Firefox();
driver.get("http://www.boc.cn/sourcedb/whpj");

elem = driver.find_element_by_css_selector(".publish > div:nth-child(3) > table:nth-child(2) > tbody:nth-child(1)");
MyData = elem.text
print(MyData)

货币名称 现汇买入价 现钞买入价 现汇卖出价 现钞卖出价 中行折算价 发布日期 发布时间
阿联酋迪拉姆 180.36 193.44 186.14 2018-09-15 21:36:54
澳大利亚元 489.58 474.37 493.18 494.26 491.65 2018-09-15 21:36:54
巴西里亚尔 157.99 172.8 162.64 2018-09-15 21:36:54
加拿大元 525.05 508.47 528.92 530.08 525.93 2018-09-15 21:36:54
瑞士法郎 707.23 685.4 712.19 713.97 707.83 2018-09-15 21:36:54
丹麦克朗 106.54 103.25 107.4 107.61 107.17 2018-09-15 21:36:54
欧元 795.37 770.66 801.24 802.83 799.49 2018-09-15 21:36:54
英镑 894.38 866.59 900.96 902.94 896.66 2018-09-15 21:36:54
港币 87.32 86.62 87.67 87.67 87.11 2018-09-15 21:36:54
印尼卢比 0.0448 0.048 0.0461 2018-09-15 21:36:54
印度卢比 8.9776 10.1236 9.4978 2018-09-15 21:36:54
日元 6.1083 5.9185 6.1532 6.1532 6.1042 2018-09-15 21:36:54
韩国元 0.6096 0.5881 0.6144 0.6368 0.6102 2018-09-15 21:36:54
澳门元 84.92 82.08 85.26 87.99 84.75 2018-09-15 21:36:54
林吉特 173.63 175.2 165.05 2018-09-15 21:36:54
挪威克朗 82.81 80.25 83.47 83.64 83.22 2018-09-15 21:36:54
新西兰元 448.09 434.26 451.23 456.76 449.62 2018-09-15 21:36:54
菲律宾比索 12.63 12.24 12.73 13.32 1

下面我们根据换行符对上面的数据进行切割，这样就得到一个列表RawData，列表元素是一个行的取值。

In [10]:
RawData = MyData.split('\n'); #Split data according to \n#
RawData

['货币名称 现汇买入价 现钞买入价 现汇卖出价 现钞卖出价 中行折算价 发布日期 发布时间',
 '阿联酋迪拉姆 180.36 193.44 186.14 2018-09-15 21:36:54',
 '澳大利亚元 489.58 474.37 493.18 494.26 491.65 2018-09-15 21:36:54',
 '巴西里亚尔 157.99 172.8 162.64 2018-09-15 21:36:54',
 '加拿大元 525.05 508.47 528.92 530.08 525.93 2018-09-15 21:36:54',
 '瑞士法郎 707.23 685.4 712.19 713.97 707.83 2018-09-15 21:36:54',
 '丹麦克朗 106.54 103.25 107.4 107.61 107.17 2018-09-15 21:36:54',
 '欧元 795.37 770.66 801.24 802.83 799.49 2018-09-15 21:36:54',
 '英镑 894.38 866.59 900.96 902.94 896.66 2018-09-15 21:36:54',
 '港币 87.32 86.62 87.67 87.67 87.11 2018-09-15 21:36:54',
 '印尼卢比 0.0448 0.048 0.0461 2018-09-15 21:36:54',
 '印度卢比 8.9776 10.1236 9.4978 2018-09-15 21:36:54',
 '日元 6.1083 5.9185 6.1532 6.1532 6.1042 2018-09-15 21:36:54',
 '韩国元 0.6096 0.5881 0.6144 0.6368 0.6102 2018-09-15 21:36:54',
 '澳门元 84.92 82.08 85.26 87.99 84.75 2018-09-15 21:36:54',
 '林吉特 173.63 175.2 165.05 2018-09-15 21:36:54',
 '挪威克朗 82.81 80.25 83.47 83.64 83.22 2018-09-15 21:36:54',
 '新西兰元 448.09 434.26 45

为了更好处理数据，我们打算建立一个数据框来保存外汇牌价数据。列表RawData的第一个元素（对应下标为0）是表格的第一行取值，也是数据框的列名。根据空格对表格第一行的取值进行切割之后得到列名的列表。

In [11]:
ColumnNames = RawData[0].split(' ')
ColumnNames

['货币名称', '现汇买入价', '现钞买入价', '现汇卖出价', '现钞卖出价', '中行折算价', '发布日期', '发布时间']

接下来新建一个数据框，列名为ColumnNames中的元素。

In [12]:
import pandas as pd
df = pd.DataFrame(columns=ColumnNames)

由于抓取的数据有缺失值，例如第一行的阿联酋迪拉姆缺了现汇买入价和现汇卖出价，因此接下来往df数据框当中填数据时，我们只把没有缺失值的行填入。判断依据是一行数据元素个数恰好等于列名元素个数。

In [13]:
p=1;
for i in range(1,len(RawData)):
    row_values = RawData[i].split(' ')
    if (len(row_values) == len(ColumnNames)):
        df.loc[p] = row_values
        p=p+1
print(df)

     货币名称   现汇买入价   现钞买入价   现汇卖出价   现钞卖出价   中行折算价        发布日期      发布时间
1   澳大利亚元  489.58  474.37  493.18  494.26  491.65  2018-09-15  21:36:54
2    加拿大元  525.05  508.47  528.92  530.08  525.93  2018-09-15  21:36:54
3    瑞士法郎  707.23   685.4  712.19  713.97  707.83  2018-09-15  21:36:54
4    丹麦克朗  106.54  103.25   107.4  107.61  107.17  2018-09-15  21:36:54
5      欧元  795.37  770.66  801.24  802.83  799.49  2018-09-15  21:36:54
6      英镑  894.38  866.59  900.96  902.94  896.66  2018-09-15  21:36:54
7      港币   87.32   86.62   87.67   87.67   87.11  2018-09-15  21:36:54
8      日元  6.1083  5.9185  6.1532  6.1532  6.1042  2018-09-15  21:36:54
9     韩国元  0.6096  0.5881  0.6144  0.6368  0.6102  2018-09-15  21:36:54
10    澳门元   84.92   82.08   85.26   87.99   84.75  2018-09-15  21:36:54
11   挪威克朗   82.81   80.25   83.47   83.64   83.22  2018-09-15  21:36:54
12   新西兰元  448.09  434.26  451.23  456.76  449.62  2018-09-15  21:36:54
13  菲律宾比索   12.63   12.24   12.73   13.32   12.66  2018-09-15  2

上面不能处理数据缺失值的做法显然不能让我们满意。使用Inspect Element（检查元素）的方法，可以找到第一行阿联酋迪拉姆的所有数据的Xpath定位地址是：/html/body/div/div[5]/div[1]/div[2]/table/tbody/tr[2]。第二行澳大利亚元的所有数据Xpath定位地址是：/html/body/div/div[5]/div[1]/div[2]/table/tbody/tr[3]。因此我们找到了定位外汇牌价数据的Xpath定位地址的规律：/html/body/div/div[5]/div[1]/div[2]/table/tbody/tr。使用Xpath定位时，可以略去前面一些标签，用双斜杠替代，不过这需要尝试一下，看是否因为省略部分标签的缘故，导致提取出我们不想要的数据。

In [14]:
#elems = driver.find_elements_by_xpath("/html/body/div/div[5]/div[1]/div[2]/table/tbody/tr");
elems = driver.find_elements_by_xpath("//div[1]/div[2]/table/tbody/tr");
for elem in elems:
    print(elem.text)

货币名称 现汇买入价 现钞买入价 现汇卖出价 现钞卖出价 中行折算价 发布日期 发布时间
阿联酋迪拉姆 180.36 193.44 186.14 2018-09-15 21:36:54
澳大利亚元 489.58 474.37 493.18 494.26 491.65 2018-09-15 21:36:54
巴西里亚尔 157.99 172.8 162.64 2018-09-15 21:36:54
加拿大元 525.05 508.47 528.92 530.08 525.93 2018-09-15 21:36:54
瑞士法郎 707.23 685.4 712.19 713.97 707.83 2018-09-15 21:36:54
丹麦克朗 106.54 103.25 107.4 107.61 107.17 2018-09-15 21:36:54
欧元 795.37 770.66 801.24 802.83 799.49 2018-09-15 21:36:54
英镑 894.38 866.59 900.96 902.94 896.66 2018-09-15 21:36:54
港币 87.32 86.62 87.67 87.67 87.11 2018-09-15 21:36:54
印尼卢比 0.0448 0.048 0.0461 2018-09-15 21:36:54
印度卢比 8.9776 10.1236 9.4978 2018-09-15 21:36:54
日元 6.1083 5.9185 6.1532 6.1532 6.1042 2018-09-15 21:36:54
韩国元 0.6096 0.5881 0.6144 0.6368 0.6102 2018-09-15 21:36:54
澳门元 84.92 82.08 85.26 87.99 84.75 2018-09-15 21:36:54
林吉特 173.63 175.2 165.05 2018-09-15 21:36:54
挪威克朗 82.81 80.25 83.47 83.64 83.22 2018-09-15 21:36:54
新西兰元 448.09 434.26 451.23 456.76 449.62 2018-09-15 21:36:54
菲律宾比索 12.63 12.24 12.73 13.32 1

继续分析elems[0]，它保存表头信息，这些信息保存在几个<th>标签下。下面的代码打印出

In [15]:
thead = elems[0].find_elements_by_tag_name('th')
ColumnNames = [];
for el in thead:
    ColumnNames = ColumnNames + [el.text]
print(ColumnNames)

['货币名称', '现汇买入价', '现钞买入价', '现汇卖出价', '现钞卖出价', '中行折算价', '发布日期', '发布时间']


继续提取信息并保存在df中。

In [16]:
import pandas as pd
df = pd.DataFrame(columns=ColumnNames)
for i in range(1,len(elems)):
    trow = elems[i].find_elements_by_tag_name('td')
    rowdata = [];
    for el in trow:
        rowdata = rowdata + [el.text]
    df.loc[i-1] = rowdata  
print(df)

      货币名称   现汇买入价   现钞买入价   现汇卖出价    现钞卖出价   中行折算价        发布日期      发布时间
0   阿联酋迪拉姆          180.36           193.44  186.14  2018-09-15  21:36:54
1    澳大利亚元  489.58  474.37  493.18   494.26  491.65  2018-09-15  21:36:54
2    巴西里亚尔          157.99            172.8  162.64  2018-09-15  21:36:54
3     加拿大元  525.05  508.47  528.92   530.08  525.93  2018-09-15  21:36:54
4     瑞士法郎  707.23   685.4  712.19   713.97  707.83  2018-09-15  21:36:54
5     丹麦克朗  106.54  103.25   107.4   107.61  107.17  2018-09-15  21:36:54
6       欧元  795.37  770.66  801.24   802.83  799.49  2018-09-15  21:36:54
7       英镑  894.38  866.59  900.96   902.94  896.66  2018-09-15  21:36:54
8       港币   87.32   86.62   87.67    87.67   87.11  2018-09-15  21:36:54
9     印尼卢比          0.0448            0.048  0.0461  2018-09-15  21:36:54
10    印度卢比          8.9776          10.1236  9.4978  2018-09-15  21:36:54
11      日元  6.1083  5.9185  6.1532   6.1532  6.1042  2018-09-15  21:36:54
12     韩国元  0.6096  0.5881  0.6144   0

最后我们关闭Selenium Webdriver。

In [17]:
driver.close();